In [20]:
#import necessary libraries

#Reprocess will create a csv file that will add the non processed tweets to another column 
#along with their preprocessed versions

import pandas as pd
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer

[nltk_data] Downloading package stopwords to C:\Users\Kirby
[nltk_data]     Wenceslao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
#Pre Processing

#Declare Stop Words

filipino_stopwords = set(
    """
akin
aking
ako
alin
am
amin
aming
ang
ano
anumang
apat
at
atin
ating
ay
bababa
bago
bakit
bawat
bilang
dahil
dalawa
dapat
din
dito
doon
gagawin
gayunman
ginagawa
ginawa
ginawang
gumawa
gusto
habang
hanggang
hindi
huwag
iba
ibaba
ibabaw
ibig
ikaw
ilagay
ilalim
ilan
inyong
isa
isang
itaas
ito
iyo
iyon
iyong
ka
kahit
kailangan
kailanman
kami
kanila
kanilang
kanino
kanya
kanyang
kapag
kapwa
karamihan
katiyakan
katulad
kaya
kaysa
ko
kong
kulang
kumuha
kung
laban
lahat
lamang
likod
lima
maaari
maaaring
maging
mahusay
makita
marami
marapat
masyado
may
mayroon
mga
minsan
mismo
mula
muli
na
nabanggit
naging
nagkaroon
nais
nakita
namin
napaka
narito
nasaan
ng
ngayon
ni
nila
nilang
nito
niya
niyang
noon
o
pa
paano
pababa
paggawa
pagitan
pagkakaroon
pagkatapos
palabas
pamamagitan
panahon
pangalawa
para
paraan
pareho
pataas
pero
pumunta
pumupunta
sa
saan
sabi
sabihin
sarili
sila
sino
siya
tatlo
tayo
tulad
tungkol
una
walang
""".split()
)

from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

search = "leni robredo bongbong marcos isko moreno domagoso manny pacman pacquiao ping lacson ernie abella leody de guzman norberto gonzales jose montemayor jr faisal mangondato"
candidatelist = search.split(" ")

#pre-process tweet input
def pre_process_tweet(tweet_input):
    
    #Step 1 - Extract Tweet from input
    #Tweet = tweet_input
    tweet = tweet_input.strip().replace("\n"," ")
    
    #Step 2 - Data Deidentification
    output = ""
    sentence = tweet.split(" ")
    for part in sentence:
        if not re.match(r"(^|[^@\w])@(\w{1,15})\b", part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"

    tweets_de_identified = output
    
    #Step 3 - URL Removal
    output = ""
    sentence = tweets_de_identified.split(" ")
    for part in sentence:
        valid = validators.url(part)

        if (not valid == True):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
                
    tweets_url_removed = output
    
    #Step 4 - Special Character Processing
    
    emoji_removed = emoji.replace_emoji(tweets_url_removed, replace="[emoji]")
    output = ""
    sentence = emoji_removed.split(" ")
    
    for part in sentence:
        if not (re.match(r"^[_\W]+$", part) or "[emoji]" in part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
    
    tweets_specialcharacters_removed = output
    
    #Step 5 - Normalization, lowercase>removediacritics>remove numerics and symbols>stopwords
    
    #lowercase the text
    lowercased_input = tweets_specialcharacters_removed.lower()

    #remove diacritics
    diacritics_removed = unidecode.unidecode(lowercased_input)

    output = ""
    sentence = diacritics_removed.split(" ")

    for part in sentence:
        part = re.sub("[^A-Za-z ]+$", "", part)
        part = re.sub("^[^A-Za-z #]+", "", part)
        if not (len(part) <= 1 or re.match(r"[^#a-zA-Z]", part) or part in english_stopwords or part in filipino_stopwords or any(part in x for x in candidatelist)):     
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"  
                
    tweets_normalized = output
    
    #Step 6 - Hashtag Processing, removing the hashtags from the tweet
    output = ""
    sentence = tweets_normalized.split(" ")

    for part in sentence:
        if not re.match(r"#(\w+)", part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
                
    tweets_hashtags_removed = output  
    #Step 7 - Tokenization
    tokenizer = WhitespaceTokenizer()
    
    output = tokenizer.tokenize(tweets_hashtags_removed)
    
    tweets_tokenized = output
    tokens = ','.join(str(s) for s in tweets_tokenized)
    
    
    return tweets_tokenized

In [22]:
#Extraction of Filipino Tweets
#Refers to opening the raw data containing the Filipino Tweets
#into the python program

data_path = "dataset.csv"
#using pandas
df = pd.read_csv(data_path)
df

,Tweet Content,Sentiment,Label
0,Dictator's son on the cusp of power in the #Ph...,Negative,Hate
1,Kung ang definition ni BongBong Marcos sa isan...,Negative,Hate
2,Leni Robredo is not being subjected by her opp...,Negative,Hate
3,@imstillsour Tuwang tuwa sila sa Pink hahaha.....,Negative,Hate
4,Kapag hindi si VP Leni Robredo ang iboboto mo ...,Negative,Hate
...,...,...,...
7675,VP LENI ROBREDO FOR 2022 https://t.co/2ahSefgmST,Neutral,Non-hate
7676,@alt_ego143 @rapplerdotcom @ramboreports hindi...,Neutral,Non-hate
7677,"I can't vote yet, but my president is Leni Rob...",Neutral,Non-hate
7678,Anyways President Leni Robredo #SaveLegendsOfT...,Neutral,Non-hate


In [23]:
tweet_list = df['Tweet Content'].values
preprocessed_list = []
for tweet in tweet_list:
    preprocessed_list.append(pre_process_tweet(tweet))

df['Preprocessed_Tweet'] = preprocessed_list

In [24]:
df

,Tweet Content,Sentiment,Label,Preprocessed_Tweet
0,Dictator's son on the cusp of power in the #Ph...,Negative,Hate,"[dictator's, cusp, power]"
1,Kung ang definition ni BongBong Marcos sa isan...,Negative,Hate,"[definition, terorista, armadong, grupo, nanan..."
2,Leni Robredo is not being subjected by her opp...,Negative,Hate,"[subjected, opponents, gender, attact, intelli..."
3,@imstillsour Tuwang tuwa sila sa Pink hahaha.....,Negative,Hate,"[tuwang, tuwa, pink, hahaha..hindi, alam, pink..."
4,Kapag hindi si VP Leni Robredo ang iboboto mo ...,Negative,Hate,"[si, vp, iboboto, years, kang, mamalasin]"
...,...,...,...,...
7675,VP LENI ROBREDO FOR 2022 https://t.co/2ahSefgmST,Neutral,Non-hate,[vp]
7676,@alt_ego143 @rapplerdotcom @ramboreports hindi...,Neutral,Non-hate,"[pangngumbinse, tawag, dyan, clearing, rlly, s..."
7677,"I can't vote yet, but my president is Leni Rob...",Neutral,Non-hate,"[can't, vote, yet, president, vice, president,..."
7678,Anyways President Leni Robredo #SaveLegendsOfT...,Neutral,Non-hate,"[anyways, president]"


In [25]:
df.to_csv('comp_dataset_multilabel.csv', index=False)